## Make Negative Samples

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import gc

In [2]:
USER, ITEM, RATING = '회원번호', 'ISBN', '평점'

In [9]:
ratings_df = pd.read_csv('book_transactions.csv', encoding='cp949').dropna(axis=0).query('ISBN != "-"')

# Add ratings column
ratings_df[RATING] = 1

ratings_df

,회원번호,일자,책제목,카테고리,작가,ISBN,출판사,출판일자,주문시간,수량,배송지,평점
0,292,20140621,유형아작 중학수학 3-2 (2014년),중고등학습서,<김순우> 등저,9788964166178,비상교육(구 비유와상징),20110801.0,23,1,서울특별시,1
1,292,20140621,개념+유형 중등수학 3-2 실력향상 파워 (2014년),중고등학습서,<박미정> 등저,9788966868674,비상교육(구 비유와상징),20140301.0,23,1,서울특별시,1
2,292,20140621,내신특강 중학 수학 3-2 (2014년),중고등학습서,<김정우> 등저,9788937898334,미래엔(대한교과서),20130530.0,23,1,서울특별시,1
3,294,20140528,액셀 월드 (ACCEL WORLD) 1,만화/라이트노벨,<카와하라 레키> 저/<HIMA> 그림/<김완> 역,9788926321379,서울문화사,20091110.0,20,1,충남,1
4,294,20140528,신약 어떤 마술의 금서목록 2,만화/라이트노벨,<카마치 카즈마> 저/<하이무라 키요타카> 그림/<김소연> 역,9788925292564,대원씨아이(단행)(대원키즈),20120115.0,20,1,충남,1
5,294,20140528,신약 어떤 마술의 금서목록 1,만화/라이트노벨,<카마치 카즈마> 저/<하이무라 키요타카> 그림/<김소연> 역,9788925288239,대원씨아이(단행)(대원키즈),20111011.0,20,1,충남,1
6,299,20140519,에몽의 영문법의 재발견,중고등학습서,<박희성> 저,9788996287001,쏠티북스,20090801.0,0,1,경상북도,1
7,299,20140519,능률 VOCA 어원편,중고등학습서,<능률영어교육연구소> 저,9788966942909,능률교육(학습),20130110.0,0,1,경상북도,1
8,300,20140518,우아한 거짓말,청소년,<김려령> 저,9788936456221,창비,20091120.0,22,1,광주광역시,1
11,308,20140512,갈매기의 꿈,청소년,<리처드 바크> 저/<신현철> 역,9788992751193,현문미디어,20070907.0,23,1,전라남도,1


In [10]:
# random split using scikit-learn
train_valid_df, test_df, _, _ = train_test_split(ratings_df, ratings_df[RATING], test_size=0.3, random_state=0)

del ratings_df
gc.collect()

0

In [11]:
# Sample n_neg negative items per positive item

import random
def negative_sampling(data, n_neg):
    item_pool = set(data[ITEM].unique())
    interact_status = (
        data.groupby(USER)[ITEM]
        .apply(set)
        .reset_index()
        .rename(columns={ITEM: ITEM + "_interacted"})
    )
    interact_status[ITEM + "_negative"] = interact_status[ITEM + "_interacted"].apply(lambda x: item_pool - x)

    users, items, ratings = [], [], []
    data = pd.merge(data, interact_status[[USER, ITEM + "_negative"]], on=USER)
    try:
        data[ITEM + "_negative"] = data[ITEM + "_negative"].apply(lambda x: random.sample(x, n_neg))
    except:
        min_num = min(map(len, list(data[ITEM + "_negative"])))
        data[ITEM + "_negative"] = data[ITEM + "_negative"].apply(lambda x: random.sample(x, min_num))
    for row in data.itertuples():
        users.append(getattr(row, USER))
        items.append(getattr(row, ITEM))
        ratings.append(float(getattr(row, RATING)))
        for i in getattr(row, ITEM + "_negative"):
            users.append(getattr(row, USER))
            items.append(i)
            ratings.append(float(0))
    return pd.DataFrame({USER: users, ITEM: items, RATING: ratings}) 

In [ ]:
# Generate negative samples for only train data
train_valid_df = negative_sampling(train_valid_df, 1)
train_valid_df

In [ ]:
# Save train & test data
from sklearn.externals import joblib 
joblib.dump((train_valid_df, test_df.loc[:,[USER, ITEM, RATING]]), 'recsys_train_test_1v1.pkl')

# For reading
#train_valid_df, test_df = joblib.load('recsys_train_test.pkl')

## End